# December 23, 2023

https://adventofcode.com/2023/day/23

In [1]:
text = f'''#.#####################
#.......#########...###
#######.#########.#.###
###.....#.>.>.###.#.###
###v#####.#v#.###.#.###
###.>...#.#.#.....#...#
###v###.#.#.#########.#
###...#.#.#.......#...#
#####.#.#.#######.#.###
#.....#.#.#.......#...#
#.#####.#.#.#########v#
#.#...#...#...###...>.#
#.#.#v#######v###.###v#
#...#.>.#...>.>.#.###.#
#####v#.#.###v#.#.###.#
#.....#...#...#.#.#...#
#.#########.###.#.#.###
#...###...#...#...#.###
###.###.#.###v#####v###
#...#...#.#.>.>.#.>.###
#.###.###.#.###.#.#v###
#.....###...###...#...#
#####################.#'''

test_text = text.split("\n")

In [67]:
fn = "data/23.txt"
with open(fn, "r") as file:
    text = file.readlines()

puzz_text = [x.strip() for x in text]

In [71]:
class Forest:
    def __init__( self, text ):
        self.text = text
        # to make one of the algos slightly simpler, we treat the start as a v. thus all segments start with a v
        self.text[0] = "#v" + self.text[0][2:]
        self.text[-1] = self.text[-1][:-2] + "v#"
        
        self.width = len(text[0])
        self.height = len(text)
        self.start = [0,1]
        self.goal = [self.height-1, self.width-2]

    def symbol( self, r, c ):
        if (c<0 or c==self.width or c<0 or c==self.height):
            return None
        return self.text[r][c]

    def next_step( self, cur, prev ):
        r,c = cur
        r0,c0 = prev

        radj_list = [-1, 0, 0, +1]
        cadj_list = [0, -1, +1, 0]

        for radj, cadj in zip(radj_list, cadj_list):
            rtry = r + radj
            ctry = c + cadj
            if (rtry == r0 and ctry == c0):
                continue
            sym = self.symbol(rtry, ctry)
            if (sym is not None) and (sym != "#"):
                break

        return rtry, ctry
    
    def slide( self, r, c ):
        sym = self.symbol(r,c)
        if sym == "<":
            return [r,c-1]
        elif sym == ">":
            return [r,c+1]
        elif sym == "^":
            return [r-1, c]
        elif sym == "v":
            return [r+1, c]
        else:
            raise BaseException(f'''{r},{c} is not a slope''')
    
    def measure_segment( self, cur ):
        # first step is sliding down the beginning slope of the segment
        prev = cur
        cur = self.slide(*cur)
        steps = 1
        sym = self.symbol(*cur)
        
        while sym == ".":
            r,c = self.next_step(cur, prev)
            steps += 1
            sym = self.symbol(r,c)
            prev = cur
            cur = [r,c]

        if cur != self.goal:
            cur = self.slide(*cur)
            steps += 1

        return steps, cur
    
    def max_dist( self, cur=None ):
        if cur is None:
            cur = self.start
            
        steps_so_far, cur = self.measure_segment( cur )
        if cur == self.goal:
            return steps_so_far
        
        # follow the slide, which is +1 step
        #prev = cur
        #cur = self.slide(*cur)
        #steps_so_far += 1

        # find possible paths forward
        to_try = []
        # try going up
        sym = self.symbol( cur[0]-1, cur[1] )
        if (sym is not None) and (sym != "v") and (sym != "#"):
            to_try.append( [cur[0]-1, cur[1]] )
        # try going down
        sym = self.symbol( cur[0]+1, cur[1] )
        if (sym is not None) and (sym != "^") and (sym != "#"):
            to_try.append( [cur[0]+1, cur[1]] )
        # try going left
        sym = self.symbol( cur[0], cur[1]-1 )
        if (sym is not None) and (sym != ">") and (sym != "#"):
            to_try.append( [cur[0], cur[1]-1] )
        # try going right
        sym = self.symbol( cur[0], cur[1]+1 )
        if (sym is not None) and (sym != "<") and (sym != "#"):
            to_try.append( [cur[0], cur[1]+1] )

        best = max( [self.max_dist(t) for t in to_try] )

        return steps_so_far + best + 1 # need to include first step of the following leg
    

    def max_dist2( self, cur=None, history=None ):
        if cur is None:
            cur = self.start
            history = [cur]
            
        steps_so_far, cur = self.measure_segment( cur )
        if cur == self.goal:
            return steps_so_far
        
        history.append(cur)
        
        # follow the slide, which is +1 step
        #prev = cur
        #cur = self.slide(*cur)
        #steps_so_far += 1

        # find possible paths forward
        to_try = []
        # try going up
        sym = self.symbol( cur[0]-1, cur[1] )
        if (sym is not None) and (sym != "#"):
            to_try.append( [cur[0]-1, cur[1]] )
        # try going down
        sym = self.symbol( cur[0]+1, cur[1] )
        if (sym is not None) and (sym != "#"):
            to_try.append( [cur[0]+1, cur[1]] )
        # try going left
        sym = self.symbol( cur[0], cur[1]-1 )
        if (sym is not None) and (sym != "#"):
            to_try.append( [cur[0], cur[1]-1] )
        # try going right
        sym = self.symbol( cur[0], cur[1]+1 )
        if (sym is not None) and (sym != "#"):
            to_try.append( [cur[0], cur[1]+1] )

        best = max( [self.max_dist(t) for t in to_try] )

        return steps_so_far + best + 1 # need to include first step of the following leg
        





### Part 1

In [72]:
test = Forest(test_text)

In [73]:
test.measure_segment( [0,1] )

(15, [5, 3])

In [74]:
test.max_dist( )

94

In [75]:
puzz = Forest(puzz_text)

In [76]:
puzz.max_dist()

2134

### Part 2